## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link
-----

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** :
    - Run the code cell below to check the performance of the algorithm on the training data.
    - Testing data are available in the directory datasets/troika/testing_data if you'd like to see the performance of the model on new data.
    
> - **Data Description** :
    - ECG signals have one channel.
    - PPG signals have two channels. we take the second channel as it poses the more challenging problem and suggested. Here in the problem i have taken second channel which is suggested.
    - Accelerometers have three channels, each corresponding to a space axis x, y, and z. I use the magnitude of these three channels as distance calculation.
    - The data is 125Hz sampled from 12 different subjects.

> - **Algorithhm Description** :
    - Fast Fourier Transformation to the PPG and IMU signals.
    - Find two strongest frequencies from the PPG data.
    - Find the peaks of PPG frequencies.
    - If no peak was found, simply get the strongest frequency and use it as the EEG estimation.
    - If the distance of magnitutes between these two frequencies is small, perform the following algorithm to choose the best peak:
    - For each peak of the Fast-Fourier Transformed IMU spectrums, see how close it is is to each peak of the PPG spectrums.
    - Choose the closest peak's PPG frequency as the estimation.
    - If there is only a single peak, use the frequency of that peak as the EEG estimation

>   - **the specific aspects of the physiology that it takes advantage of** : PPG signals can be used for measuring heart rate. Capillaries in the wrist fill with blood when the ventricles contract, when the blood passes light emitted by the PPG sensor is absorbed by red blood cells in these capillaries and the photodetector will see the cut in reflected light. Change in light measures and this oscillating waveform is the pulse rate.

>   - **describtion of the algorithm outputs** : 
    - Outputs: the estimated frequency (in BPM) and the confidence score of that prediction.

>   - **caveats on algorithm outputs** : The confidence rate is only calculated based on the magnitude of a small area that contains the estimated spectral frequency relative to the sum magnitude of the entire spectrum. 

>   - **common failure modes** : When the PPG picks a higher frequency signal that is not from the heart rate. This is possible due to hand movements, arm movement, alivation. To overcome with this, the accelerations measurmnet use in the algorithm.

> - **Algorithm Performance** :
    - The performance was calculated by calculating the mean absolute error between the HR estimation and the reference HR from the ECG sensors at 90% availability. Put another way, 90% of the best estimates according to the algorithm's confidence scores.
    - The error of the model in the testing data is 0.60

The algorithm requires its parameters to have the exact values that are currently used as reference to the desired low error score. To improve performance, a machine learning approach can be usefull, with also more test subjects performing various kinds of different activities. 


### Code

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.stats
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

fs = 125
window_len_s = 10
window_shift_s = 2
past_window = 3
pass_band = (40/60.0, 240/60.0)
multiplier = 4
ppg_mag_height = 0.55
acc_mag_height = 0.3
ppg_min_dist = 0.2
num_best = 2
acc_num_best_arg = 2

def bandpass_filter(signal):
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def get_indxs(sig_len, ref_len):
    
    if ref_len < sig_len:
        n = ref_len
    else:
        n = sig_len
    
    start_indxs = (np.cumsum(np.ones(n) * fs * window_shift_s) - fs * window_shift_s).astype(int)
    end_indxs = start_indxs + window_len_s * fs
    
    return (start_indxs, end_indxs)

def Predict(ppg, accx, accy, accz):
    
    ppg = bandpass_filter(ppg)        
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
        
    n = len(ppg) * multiplier
    freqs = np.fft.rfftfreq(n, 1/fs)

    fft = np.abs(np.fft.rfft(ppg, n))
    fft[freqs <= pass_band[0]] = 0.0
    fft[freqs >= pass_band[1]] = 0.0

    acc_l2 = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)

    acc_fft = np.abs(np.fft.rfft(acc_l2, n))
    acc_fft[freqs <= pass_band[0]] = 0.0
    acc_fft[freqs >= pass_band[1]] = 0.0


    peaks, _ = scipy.signal.find_peaks(fft, height=ppg_mag_height*np.max(fft), distance=1)
    max_ppg_fs = freqs[peaks]

    acc_peaks, _ = scipy.signal.find_peaks(acc_fft, height=acc_mag_height*np.max(acc_fft), distance=50)
    max_acc_fs = freqs[acc_peaks]
    max_acc_ids = []
    
    if len(max_ppg_fs) == 0:
        best_fit = freqs[np.argmax(fft)]
        
    elif len(max_ppg_fs) > 1:
        max_ppg_ids = np.argpartition(fft[peaks], -num_best)[-num_best:]
        max_ppg_ids = max_ppg_ids[np.argsort(-fft[peaks][max_ppg_ids])]
        max_ppg_fs = max_ppg_fs[max_ppg_ids]
    
        best_fit = max_ppg_fs[0]
        
        ppg_best_distance = fft[peaks][max_ppg_ids[0]] - fft[peaks][max_ppg_ids[1]] # Here change
        ppg_best_distance = ppg_best_distance/1000
        ppg_min_dist_val = fft[peaks][max_ppg_ids[0]] * ppg_min_dist
        if len(acc_peaks) > 0 and ppg_best_distance < ppg_min_dist_val:
            if len(acc_peaks) < acc_num_best_arg:
                acc_num_best = len(acc_peaks)
            else:
                acc_num_best = acc_num_best_arg
            max_acc_ids = np.argpartition(acc_fft[acc_peaks], -acc_num_best)[-acc_num_best:]
            max_acc_ids = max_acc_ids[np.argsort(-acc_fft[acc_peaks][max_acc_ids])]
            max_acc_fs = max_acc_fs[max_acc_ids] 
            
            if max_acc_fs[0] == best_fit:
                if max_ppg_fs[0]< max_ppg_fs[1]:
                    best_fit = max_ppg_fs[1]

            closest_i = 0
            closest_dist = np.inf
            for i in range(num_best):
                dist = 0
                if closest_dist > dist:
                    for j in range(acc_num_best):
                        dist_j = np.abs(max_acc_fs[j] - max_ppg_fs[i])
                        dist += dist_j

                    closest_dist = dist
                    closest_i = i
            best_fit = max_ppg_fs[closest_i]
    else:
        best_fit = max_ppg_fs[0]
        
    fs_win = 45  / 60.0
    fs_win_e = (freqs >= best_fit - fs_win) & (freqs <= best_fit +fs_win)
    conf = np.sum(fft[fs_win_e])/np.sum(fft)
    
    est = best_fit * 60
    
    return (est, conf)


def RunPulseRateAlgorithm(data_fl, ref_fl, log_vars=True):
    """
    Run Pulse Rate Algorithm
    
    Loads data and model, then calculate errors and confidence rates from the data.
    
    Args:
        data_fl: (string) Path to data file (MATLAB data)
        ref_fl: (string) Path to reference data file (MATLAB data)
        log_vars: (bool) If True, log some variables in the `log` object for further analysis
        
    Returns:
        (np.array) Error scores
        (np.array) Confidence rates
    """

    signals = scipy.io.loadmat(data_fl)['sig']
    references = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    errors = []
    confidence = []
    estimations = []
    
    start_idxs, end_idxs = get_indxs(signals.shape[1], len(references))
    
    for i, start_idx in enumerate(start_idxs):
        end_idx = end_idxs[i]
        ref = references[i]
        
        ppg = signals[2, start_idx:end_idx]
        accx = signals[3, start_idx:end_idx]
        accy = signals[4, start_idx:end_idx]
        accz = signals[5, start_idx:end_idx]
        
        est, conf = Predict(ppg, accx, accy, accz)

        error = np.abs(est - ref)

        errors.append(error)
        confidence.append(conf)
        estimations.append(est)
    
    return np.array(errors), np.array(confidence)
Evaluate()

20.168551153481324